In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import os 
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)

/home/bumjin/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [53]:
exp_name = "exp_1_not"
dir_path = f"results/{exp_name}/General_Binary"
models = [
          'openai--gpt-4o-mini', 
          'llama3_1_instruct--70b', 
          'llama3_1--8b', 
          'gemma2--9b', 
          'exaone--8b', 
          'qwen2--7b'
          ]
result_dfs = []
for model in models:
    result_df = pd.read_csv(os.path.join(dir_path, f"{model}.csv"))
    result_df['model'] = model
    result_dfs.append(result_df)

def mv_func(x):
    values = ['must', 'should', 'ought']
    for v in values:
        if v in x:
            return v 
    if "has to" in x or "have to" in x:
        return "have to"
    return "none"

def process_model_output(x):
    x =str(x)
    if "1" in x and "0" in x:
        return 2
    elif "1" in x:
        return 1
    elif "0" in x:
        return 0
    else:
        return -1 
result_df = pd.concat(result_dfs)
result_df['mv'] = result_df['input'].apply(mv_func)
result_df['model_output'] =  result_df['model_output_raw'].apply(process_model_output)
result_df.head()

targets = ['input_type', 'model', 'model_output']
groupby = targets[:-1]
result_averaged = result_df[targets].groupby(groupby).mean().reset_index()
result_averaged
new_df = pd.DataFrame(columns=['type'] + models)
for input_type in ['none', 'strong', 'weak']:
    new_row = [input_type]
    for model in models:
        temp = result_averaged[result_averaged['input_type'] == input_type]
        temp = temp[temp['model'] == model]
        new_row.append(np.round(temp['model_output'].item(), 2))
    new_df.loc[len(new_df)] = new_row
new_df
new_df.to_csv(f'averaged_{exp_name}.csv', index=False)

,type,openai--gpt-4o-mini,llama3_1_instruct--70b,llama3_1--8b,gemma2--9b,exaone--8b,qwen2--7b
0,none,0.13,0.08,0.99,0.43,-0.09,0.19
1,strong,0.89,0.70,1.00,0.86,-0.04,0.82
2,weak,0.52,0.21,0.98,0.66,-0.17,0.37
